In [1]:
# Import modules
from pymatgen.symmetry.groups import SpaceGroup
from pymatgen.io.vasp import Poscar
from pymatgen.core import Structure
from pymatgen.io.vasp import Xdatcar
import numpy as np
from collections import Counter
from site_analysis.polyhedral_site import PolyhedralSite
from polyhedral_analysis.configuration import Configuration
from polyhedral_analysis.polyhedra_recipe import PolyhedraRecipe
from site_analysis.atom import atoms_from_species_string
from site_analysis.trajectory import Trajectory

/Users/goldmann/opt/anaconda3/lib/python3.9/site-packages/pymatgen/core/__init__.py:49: UserWarning: Error loading .pmgrc.yaml: [Errno 2] No such file or directory: '/Users/goldmann/.pmgrc.yaml'. You may need to reconfigure your yaml file.
  warnings.warn(f"Error loading .pmgrc.yaml: {ex}. You may need to reconfigure your yaml file.")


ModuleNotFoundError: No module named 'polyhedral_analysis'

In [2]:
# Read XDATCAR
xdat = Xdatcar('XDATCAR')

FileNotFoundError: [Errno 2] No such file or directory: 'XDATCAR'

In [35]:
# Create structure for each Wyckoff site
all_li_structure = Structure.from_file('all_li_structure.cif')
lattice = all_li_structure.lattice

li1 = Structure.from_spacegroup(sg='C2/m', lattice = lattice, species=['Li'], coords=[[0.50000, 0.00000, 0.50000]]) # octahedral Li not in Sc layer, but in line with Sc (2d)
li2 = Structure.from_spacegroup(sg='C2/m', lattice = lattice, species=['Li'], coords=[[0.50000, 0.83333, 0.00000]]) # octahedral Li in Sc layer (4g)
li3 = Structure.from_spacegroup(sg='C2/m', lattice = lattice, species=['Li'], coords=[[0.00000, 0.16667, 0.50000]]) # octahedral Li not in Sc layer (4h)
li4 = Structure.from_spacegroup(sg='C2/m', lattice = lattice, species=['Li'], coords=[[0.87500, 0.50000, 0.12500]]) # tetrahedral Li in Sc layer and in line with Sc (4i_1)
li5 = Structure.from_spacegroup(sg='C2/m', lattice = lattice, species=['Li'], coords=[[0.87500, 0.00000, 0.62500]]) # tetrahedral Li not in Sc layer, but in line with Sc (4i_2)
li7 = Structure.from_spacegroup(sg='C2/m', lattice = lattice, species=['Li'], coords=[[0.87500, 0.16667, 0.12500]]) # tetrahedral Li in Sc layer, but not in line with Sc (8j_1)
li6 = Structure.from_spacegroup(sg='C2/m', lattice = lattice, species=['Li'], coords=[[0.12500, 0.33333, 0.37500]]) # tetrahedral Li not in Sc layer and not in line with Sc (8j_2)

li_site_structures = [li1, li2, li3, li4, li5, li6, li7]

for strc in li_site_structures:
    strc.make_supercell([3,2,3])

/home/ben/.local/lib/python3.8/site-packages/pymatgen/io/cif.py:1155: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [36]:
# Create list of all polyhedral sites
li_site_structures = [li1, li2, li3, li4, li5, li6, li7]
li_labels = ['2d', '4g', '4h', '4i_1', '4i_2', '8j_1', '8j_2']
all_li_sites = []

for li_site_structure, li_label in zip(li_site_structures, li_labels):
    # Read initial strucutre:
    initial_structure = Structure.from_file('POSCAR')
    
    # Append atoms for Wyckoff site to initial structure
    for site in li_site_structure:
        initial_structure.append(site.species, site.frac_coords)
    
    # Create list of atoms that describe Wyckoff site
    li_indices = []
    for i in range(360, len(initial_structure)):
        li_indices.append(i)
    
    # Define polyhedral recipe
    recipe = PolyhedraRecipe(method='distance cutoff', 
                          coordination_cutoff=3.2, 
                          central_atoms=li_indices,
                          vertex_atoms='Cl')
    
    # Create polyhedral configuration
    config = Configuration( structure=initial_structure, recipes=[recipe] )
    
    # Create polyhedral site and add to list
    for i in config.polyhedra:
        all_li_sites.append(PolyhedralSite(i.vertex_indices, label=li_label))

In [37]:
# Create trajectory
initial_structure = Structure.from_file('POSCAR')
atoms = atoms_from_species_string(initial_structure, 'Li')
trajectory = Trajectory(sites=all_li_sites, atoms=atoms)

# Load structures into trajectory
xdat_structures = xdat.structures
for timestep, s in enumerate(xdat_structures[::10]):
    trajectory.append_timestep(s, t=timestep)

# Site occupation analysis

In [38]:
# Get site occupancies
n_timesteps = len(trajectory.timesteps)
c_sites = Counter(trajectory.site_labels())
c = Counter()
p_occ = {}
for site in trajectory.sites:
    c[site.label] += len([ 1 for ts in site.trajectory if len(ts)>0 ])
for k, v in c.items():
    p_occ[k] = v / c_sites[k] / n_timesteps
p_occ

{'2d': 0.33467896962706656,
 '4g': 0.8795655517108805,
 '4h': 0.2692714340638216,
 '4i_1': 0.0,
 '4i_2': 0.002499038831218762,
 '8j_1': 0.022347174163783158,
 '8j_2': 2.4029219530949632e-05}

# Residency analysis

In [39]:
li_labels = ['2d', '4g', '4h', '4i_1', '4i_2', '8j_1', '8j_2']

for label in li_labels:
    r_times = []
    for i in trajectory.sites:
        if i.label == label:
            counter = 0
            current_atom = []
            for atom in i.trajectory:
                if len(atom) == 1:
                    if current_atom == []:
                        counter = counter + 1
                        current_atom = atom[0]
                    if current_atom != []:
                        if current_atom == atom[0]:
                            counter = counter + 1
                        elif current_atom != atom[0]:
                            r_times.append(counter-1)
                            counter = 0
                            current_atom = atom[0]
                if len(atom) == 0:
                    if current_atom != []:
                        r_times.append(counter-1)
                        counter = 0
                        current_atom = []
    print(label, np.mean(r_times), "ps")

2d 13.703389830508474 ps
4g 42.747058823529414 ps
4h 12.664179104477611 ps
4i_1 nan ps
4i_2 1.0196078431372548 ps
8j_1 1.6570397111913358 ps
8j_2 1.0 ps


/home/ben/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ben/.local/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [34]:
len(li2)

4

In [41]:
len(trajectory)

289